# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.68,64,10,1.79,NZ,1701746954
1,1,college,64.8569,-147.8028,-17.53,80,75,2.57,US,1701746955
2,2,bilibino,68.0546,166.4372,-25.87,100,100,1.16,RU,1701746956
3,3,bethel,41.3712,-73.4140,5.84,76,92,0.89,US,1701746958
4,4,constantia,44.1833,28.6500,1.50,68,73,4.52,RO,1701746959


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
plot1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles="OSM", size="Humidity", frame_height=900, frame_width=900, color="City", alpha=0.5,\
                                  xlabel="x", ylabel="y")

# Display the map
plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[(city_data_df['Max Temp'] >=20.00) &  (city_data_df['Max Temp'] <=27.00)]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,ixtapa,20.7000,-105.2000,24.01,83,75,0.00,MX,1701746959
14,14,pontes e lacerda,-15.2261,-59.3353,22.45,98,100,0.33,BR,1701746968
17,17,mount gambier,-37.8333,140.7667,24.18,36,56,5.51,AU,1701746970
24,24,adamstown,-25.0660,-130.1015,22.69,78,100,5.58,PN,1701746976
26,26,port mathurin,-19.6833,63.4167,25.81,81,28,7.72,MU,1701746978
...,...,...,...,...,...,...,...,...,...,...
569,569,wanning,18.8003,110.3967,24.94,77,85,3.84,CN,1701747542
570,570,waimea,20.0231,-155.6717,21.12,95,30,2.24,US,1701747543
574,574,urangan,-25.3000,152.9000,26.79,67,4,7.49,AU,1701747548
587,587,forster,-32.1667,152.5167,26.32,71,0,5.66,AU,1701747561


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[['City', 'Country', "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,ixtapa,MX,20.7000,-105.2000,83,
14,pontes e lacerda,BR,-15.2261,-59.3353,98,
17,mount gambier,AU,-37.8333,140.7667,36,
24,adamstown,PN,-25.0660,-130.1015,78,
26,port mathurin,MU,-19.6833,63.4167,81,
...,...,...,...,...,...,...
569,wanning,CN,18.8003,110.3967,77,
570,waimea,US,20.0231,-155.6717,95,
574,urangan,AU,-25.3000,152.9000,67,
587,forster,AU,-32.1667,152.5167,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {"categories": categories,
          "apiKey": geoapify_key,
          "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # print(latitude)
    # print(longitude)
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # pprint(name_address)
    # break
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ixtapa - nearest hotel: Ma. Cristina
pontes e lacerda - nearest hotel: Hotel Avenida
mount gambier - nearest hotel: The Old Mount Gambier Gaol
adamstown - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
blackmans bay - nearest hotel: Villa Howden
hawaiian paradise park - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
las tablas - nearest hotel: Hotel Don Jesus
salinopolis - nearest hotel: Hotel Salinópolis
san antonio de pale - nearest hotel: No hotel found
nabire - nearest hotel: Hotel Anggrek
soio - nearest hotel: Hotel Maria Teres
arraial do cabo - nearest hotel: No hotel found
west end - nearest hotel: The Inn at Old Bahama Bay
sal rei - nearest hotel: Hotel Marine Club
copala - nearest hotel: No hotel found
talodi - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
hard bargain - nearest hotel: No hotel found
meulaboh - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
5,ixtapa,MX,20.7000,-105.2000,83,Ma. Cristina
14,pontes e lacerda,BR,-15.2261,-59.3353,98,Hotel Avenida
17,mount gambier,AU,-37.8333,140.7667,36,The Old Mount Gambier Gaol
24,adamstown,PN,-25.0660,-130.1015,78,No hotel found
26,port mathurin,MU,-19.6833,63.4167,81,Escale Vacances
...,...,...,...,...,...,...
569,wanning,CN,18.8003,110.3967,77,No hotel found
570,waimea,US,20.0231,-155.6717,95,No hotel found
574,urangan,AU,-25.3000,152.9000,67,Ramada
587,forster,AU,-32.1667,152.5167,71,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
plot2 = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_height=900, frame_width=900, color="City", alpha=0.5, \
                               hover_cols=["Hotel Name","Country"], xlabel="x", ylabel="y")

# Display the map
plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)